In [1]:
import pandas as pd
import numpy as np

# Getting Track

In [45]:
playlist = [ '2BW6pZAtfOC0Uau5TDTXga', '1ZKdvf5yXvwVyzgznVz2Nl', '2Klr3jxOoym0CpdUsQPzRF', '5xIZRYHAJ3ig68dqwInhmU', '3IpDoXyKOPgxJvUJYsagyM', '1gd8rPLzo3bjk1yyQce0h7', '1nAFuLv3VOaQ85D7BlVJj5', '693upujRZBq6WNH30wwFvT', '37i9dQZF1DXdFesNN9TzXT', '6YbE3TpVwsW57jTGyCWMQe', '40Vyfk8Cv5RUDWdZxNpTbr', '37i9dQZF1DWVDvBpGQbzXj', '1qf2SGnCWGUlAVv23iuimi', '37i9dQZF1DWYztMONFqfvX', '2FI5law8pLy8soTgAxsKWu', '0zc6Hq9OIAengtGG6a3lfs', '0a2NupngERezPoIn21zblj', '3ba6w8ldKCmJN8IxI8ClXm', '6wVv6RynspgsuFVpzUigDu' ,  '37i9dQZF1DXdpQPPZq3F7n' , '7sTkp2X5Aq84v9w9UtfkaF' , '2Kj5NUtVetggUDHPIGC9U7' , '6NZrZvFaHYRaJ2HE8pp07K' , '5cfIZWLbFP5bn9C7UfxLbI' , '37i9dQZF1DWXtlo6ENS92N' ,'0S7boBN3rPfo2F1DXX5U7k', '37i9dQZF1DWZNJXX2UeBij', '0udjmvwKO4SQcXzVL9Kv1m',  '2QnLDxeZMzIoCno54I9vKj', '4nqbYFYZOCospBb4miwHWy', '37i9dQZF1DX8xfQRRX1PDm']

In [1]:
import requests
import base64

client_id = 'a'
client_secret = 'b'

In [39]:
def get_toke():
    # Get the token
    auth_string=client_id+":"+client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes),"utf=8")
    url = "https://accounts.spotify.com/api/token"
    data = {
        "grant_type": "client_credentials"
    }
    headers = {
        "Authorization": "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    response = requests.post(url, data=data, headers=headers)
    data = response.json()
    token = data['access_token']
    return token

In [40]:
def get_auth_header(token):
    return {"Authorization":"Bearer "+token}

In [41]:
headers = get_auth_header(get_toke())

In [42]:
params = {
    'limit': 100,
    'offset': 0
}

In [50]:
df = pd.DataFrame(columns=['song_name','track_spotify_id','spotify_track_link','thumbnail_link','artist_name','artist_id','artist_link'])

In [46]:
data_list = []

In [52]:
for i in playlist:
    endpoint = f'https://api.spotify.com/v1/playlists/{i}/tracks'
    
    while True:
        response = requests.get(endpoint, headers=headers, params=params)
        data = response.json()
        for item in data['items']:
            track = item['track']
            try:
                song_name = track['name']
            except KeyError:
                song_name = np.nan

            try:
                track_spotify_id = track['id']
            except KeyError:
                track_spotify_id = np.nan

            try:
                spotify_track_link = track['external_urls']['spotify']
            except KeyError:
                spotify_track_link = np.nan

            try:
                thumbnail_link = track['album']['images'][0]['url']
            except (KeyError, IndexError):
                thumbnail_link = np.nan

            try:
                artist_name = track['artists'][0]['name']
            except (KeyError, IndexError):
                artist_name = np.nan

            try:
                artist_id = track['artists'][0]['id']
            except (KeyError, IndexError):
                artist_id = np.nan

            try:
                artist_link = track['artists'][0]['external_urls']['spotify']
            except (KeyError, IndexError):
                artist_link = np.nan

            data_list.append({
                'song_name': song_name,
                'track_spotify_id': track_spotify_id,
                'spotify_track_link': spotify_track_link,
                'thumbnail_link': thumbnail_link,
                'artist_name': artist_name,
                'artist_id': artist_id,
                'artist_link': artist_link
            })

        # Check if there are more pages
        if data['next']:
            endpoint = data['next']
        else:
            break

In [53]:
df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)

In [54]:
df.shape

(1884, 7)

In [78]:
df.isnull().sum()

song_name              0
track_spotify_id       0
spotify_track_link     0
thumbnail_link        76
artist_name            0
artist_id              0
artist_link            0
acousticness           0
danceability           0
duration_ms            0
energy                 0
speechiness            0
tempo                  0
time_signature         0
valence                0
key                    0
instrumentalness       0
liveness               0
loudness               0
mode                   0
id                     0
dtype: int64

In [59]:
df.duplicated().sum()

np.int64(363)

In [61]:
df.drop_duplicates(keep='first',inplace=True)

In [62]:
df.to_csv(r'C:\Users\tarun\Desktop\song-recommender-system\data\first.csv', index=False)

In [11]:
df.isnull().sum()

song_name             76
track_spotify_id       0
spotify_track_link     0
thumbnail_link        76
artist_name           76
artist_id              0
artist_link            0
dtype: int64

In [13]:
df.shape

(1521, 7)

In [15]:
df.dropna().shape

(1445, 7)

In [17]:
df=df.dropna()

In [18]:
df.isnull().sum()

song_name             0
track_spotify_id      0
spotify_track_link    0
thumbnail_link        0
artist_name           0
artist_id             0
artist_link           0
dtype: int64

In [19]:
df.to_csv(r'C:\Users\tarun\Desktop\song-recommender-system\data\first.csv', index=False)

# Getting info about tracks

In [60]:
df=pd.read_csv(r'C:\Users\tarun\Desktop\song-recommender-system\data\first.csv')

In [64]:
df

,song_name,track_spotify_id,spotify_track_link,thumbnail_link,artist_name,artist_id,artist_link,acousticness,danceability,duration_ms,energy,speechiness,tempo,time_signature,valence,key,instrumentalness,liveness,loudness,mode
0,Sadka,4dPeCbg2Op44s9kwyMS3ZT,https://open.spotify.com/track/4dPeCbg2Op44s9k...,https://i.scdn.co/image/ab67616d0000b2731992dd...,Vishal-Shekhar,6Mv8GjQa7LKUGCAqa9qqdb,https://open.spotify.com/artist/6Mv8GjQa7LKUGC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Surili Akhiyon Wale,1la0ICQBLfO0lyqsWNwG7l,https://open.spotify.com/track/1la0ICQBLfO0lyq...,https://i.scdn.co/image/ab67616d0000b2730274bd...,Rahat Fateh Ali Khan,3OLGltG8UPIea8sA4w0yg0,https://open.spotify.com/artist/3OLGltG8UPIea8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hona Tha Pyar,0KSOLEBixnBYIKNHF1VbzF,https://open.spotify.com/track/0KSOLEBixnBYIKN...,https://i.scdn.co/image/ab67616d0000b27371ef95...,Atif Aslam,2oSONSC9zQ4UonDKnLqksx,https://open.spotify.com/artist/2oSONSC9zQ4Uon...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Zehnaseeb,3xF15SqY32Mno3UUMBirhU,https://open.spotify.com/track/3xF15SqY32Mno3U...,https://i.scdn.co/image/ab67616d0000b2732025db...,Vishal-Shekhar,6Mv8GjQa7LKUGCAqa9qqdb,https://open.spotify.com/artist/6Mv8GjQa7LKUGC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rait Zara Si,3gOXq3FPPesm7VLPKmef6j,https://open.spotify.com/track/3gOXq3FPPesm7VL...,https://i.scdn.co/image/ab67616d0000b2736caa52...,Arijit Singh,4YRxDV8wJFPHPTeXepOstw,https://open.spotify.com/artist/4YRxDV8wJFPHPT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,Pyar Hua Chupke Se,3MV6JBYaxeTuzF0mIr7bzg,https://open.spotify.com/track/3MV6JBYaxeTuzF0...,https://i.scdn.co/image/ab67616d0000b27313e0f6...,Kavita Krishnamurthy,6WPmTGeeoymoVlXVtsCwz7,https://open.spotify.com/artist/6WPmTGeeoymoVl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1441,Kasto Mazza,7uTi48q5iwaHKcuC26I1tX,https://open.spotify.com/track/7uTi48q5iwaHKcu...,https://i.scdn.co/image/ab67616d0000b273db8153...,Sonu Nigam,1dVygo6tRFXC8CSWURQJq2,https://open.spotify.com/artist/1dVygo6tRFXC8C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1442,"Aadat - From ""Kalyug""",5VrLz2NUTLbPzgjuVOjxUx,https://open.spotify.com/track/5VrLz2NUTLbPzgj...,https://i.scdn.co/image/ab67616d0000b273a42a1a...,Atif Aslam,2oSONSC9zQ4UonDKnLqksx,https://open.spotify.com/artist/2oSONSC9zQ4Uon...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1443,"Ve Kamleya (From ""Rocky Aur Rani Kii Prem Kaha...",6EivvaEepVTjmDde43eGiB,https://open.spotify.com/track/6EivvaEepVTjmDd...,https://i.scdn.co/image/ab67616d0000b27339890c...,Pritam,1wRPtKGflJrBx9BmLsSwlU,https://open.spotify.com/artist/1wRPtKGflJrBx9...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
new_columns=[
    'acousticness', 'danceability', 'duration_ms', 'energy',
    'speechiness', 'tempo', 'time_signature', 'valence', 'key',
    'instrumentalness', 'liveness', 'loudness', 'mode'
]

In [63]:
for i in new_columns:
    df[i]=np.nan

In [65]:
for index, row in df.iterrows():
    track_spotify_id=row['track_spotify_id']
    url = f'https://api.spotify.com/v1/audio-features/{track_spotify_id}'
    headers = get_auth_header(get_toke())
    response = requests.get(url, headers=headers)
    data = response.json()
    df.at[index,'acousticness'] = data.get('acousticness', np.nan)
    df.at[index,'acousticness']= data.get('acousticness', np.nan),
    df.at[index,'danceability']= data.get('danceability', np.nan),
    df.at[index,'duration_ms']= data.get('duration_ms', np.nan),
    df.at[index,'energy']= data.get('energy', np.nan),
    df.at[index,'speechiness']= data.get('speechiness', np.nan),
    df.at[index,'tempo']= data.get('tempo'),
    df.at[index,'time_signature']= data.get('time_signature', np.nan),
    df.at[index,'valence']= data.get('valence'),
    df.at[index,'key']= data.get('key'),
    df.at[index,'instrumentalness']= data.get('instrumentalness', np.nan),
    df.at[index,'liveness']= data.get('liveness', np.nan),
    df.at[index,'loudness']= data.get('loudness', np.nan),
    df.at[index,'mode']= data.get('mode', np.nan)
        

In [71]:
df.to_csv(r'C:\Users\tarun\Desktop\song-recommender-system\data\data_with_audio_feature.csv',index=False)

# Classifing the track into various geners with of Chatgpt

In [2]:
df=pd.read_csv(r'C:\Users\tarun\Desktop\song-recommender-system\data\data_with_audio_feature.csv')

In [3]:
df

,song_name,track_spotify_id,spotify_track_link,thumbnail_link,artist_name,artist_id,artist_link,acousticness,danceability,duration_ms,energy,speechiness,tempo,time_signature,valence,key,instrumentalness,liveness,loudness,mode
0,Sadka,4dPeCbg2Op44s9kwyMS3ZT,https://open.spotify.com/track/4dPeCbg2Op44s9k...,https://i.scdn.co/image/ab67616d0000b2731992dd...,Vishal-Shekhar,6Mv8GjQa7LKUGCAqa9qqdb,https://open.spotify.com/artist/6Mv8GjQa7LKUGC...,0.232,0.678,343587.0,0.547,0.0286,143.892,4.0,0.579,7.0,0.000176,0.0785,-6.890,1.0
1,Surili Akhiyon Wale,1la0ICQBLfO0lyqsWNwG7l,https://open.spotify.com/track/1la0ICQBLfO0lyq...,https://i.scdn.co/image/ab67616d0000b2730274bd...,Rahat Fateh Ali Khan,3OLGltG8UPIea8sA4w0yg0,https://open.spotify.com/artist/3OLGltG8UPIea8...,0.407,0.517,331747.0,0.561,0.0303,75.977,4.0,0.448,4.0,0.000000,0.0853,-5.777,1.0
2,Hona Tha Pyar,0KSOLEBixnBYIKNHF1VbzF,https://open.spotify.com/track/0KSOLEBixnBYIKN...,https://i.scdn.co/image/ab67616d0000b27371ef95...,Atif Aslam,2oSONSC9zQ4UonDKnLqksx,https://open.spotify.com/artist/2oSONSC9zQ4Uon...,0.571,0.687,221987.0,0.542,0.0307,102.994,4.0,0.782,4.0,0.030700,0.1180,-8.689,1.0
3,Zehnaseeb,3xF15SqY32Mno3UUMBirhU,https://open.spotify.com/track/3xF15SqY32Mno3U...,https://i.scdn.co/image/ab67616d0000b2732025db...,Vishal-Shekhar,6Mv8GjQa7LKUGCAqa9qqdb,https://open.spotify.com/artist/6Mv8GjQa7LKUGC...,0.361,0.623,217440.0,0.595,0.0331,78.029,4.0,0.418,7.0,0.000136,0.3530,-6.342,1.0
4,Rait Zara Si,3gOXq3FPPesm7VLPKmef6j,https://open.spotify.com/track/3gOXq3FPPesm7VL...,https://i.scdn.co/image/ab67616d0000b2736caa52...,Arijit Singh,4YRxDV8wJFPHPTeXepOstw,https://open.spotify.com/artist/4YRxDV8wJFPHPT...,0.781,0.563,291462.0,0.495,0.0362,125.998,4.0,0.244,2.0,0.000000,0.0837,-10.272,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,Pyar Hua Chupke Se,3MV6JBYaxeTuzF0mIr7bzg,https://open.spotify.com/track/3MV6JBYaxeTuzF0...,https://i.scdn.co/image/ab67616d0000b27313e0f6...,Kavita Krishnamurthy,6WPmTGeeoymoVlXVtsCwz7,https://open.spotify.com/artist/6WPmTGeeoymoVl...,0.639,0.351,313787.0,0.575,0.0286,101.025,4.0,0.609,2.0,0.000068,0.1540,-5.949,1.0
1441,Kasto Mazza,7uTi48q5iwaHKcuC26I1tX,https://open.spotify.com/track/7uTi48q5iwaHKcu...,https://i.scdn.co/image/ab67616d0000b273db8153...,Sonu Nigam,1dVygo6tRFXC8CSWURQJq2,https://open.spotify.com/artist/1dVygo6tRFXC8C...,0.549,0.639,279589.0,0.380,0.0379,123.953,4.0,0.755,5.0,0.000041,0.2640,-15.846,1.0
1442,"Aadat - From ""Kalyug""",5VrLz2NUTLbPzgjuVOjxUx,https://open.spotify.com/track/5VrLz2NUTLbPzgj...,https://i.scdn.co/image/ab67616d0000b273a42a1a...,Atif Aslam,2oSONSC9zQ4UonDKnLqksx,https://open.spotify.com/artist/2oSONSC9zQ4Uon...,0.136,0.624,333740.0,0.816,0.0394,102.954,4.0,0.324,0.0,0.000583,0.2450,-4.047,0.0
1443,"Ve Kamleya (From ""Rocky Aur Rani Kii Prem Kaha...",6EivvaEepVTjmDde43eGiB,https://open.spotify.com/track/6EivvaEepVTjmDd...,https://i.scdn.co/image/ab67616d0000b27339890c...,Pritam,1wRPtKGflJrBx9BmLsSwlU,https://open.spotify.com/artist/1wRPtKGflJrBx9...,0.810,0.468,246989.0,0.658,0.0668,87.607,4.0,0.534,2.0,0.033300,0.1200,-7.600,1.0


In [ ]:
pip install openai==0.28

In [4]:
import openai

In [ ]:
openai.api_key = 'c'

In [6]:
categories=[
    'Romantic Ballad', 'Dance Pop', 'Sad Ballad', 'Folk Fusion', 'Classical Fusion',
    'Qawwali', 'Sufi', 'Party Anthem', 'Patriotic', 'Rock Fusion', 'Item Number',
    'Lyrical Rap', 'Devotional', 'Ghazal', 'Retro', 'Comedy', 'Motivational',
    'Wedding', 'Festival'
]

In [7]:
for i in categories:
    df[i]=np.nan

In [8]:
def classify_song(song_name):
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Change the model to GPT-4
        messages=[
            {"role": "system", "content": "You are a music classification assistant."},
            {"role": "user", "content": f"Classify the song '{song_name}' into one of these categories: {', '.join(categories)}."}
        ]
    )
    return response.choices[0].message['content']

In [9]:
for index, row in df.iterrows():
    classification = classify_song(row['song_name'])
    for category in categories:
        if category in classification:
            df.at[index, category] = 1

In [22]:
df.to_csv(r'C:\Users\tarun\Desktop\song-recommender-system\data\data_with_genre.csv', index=False)

# Getting images 

In [3]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

In [6]:
df=pd.read_csv(r'C:\Users\tarun\Desktop\song-recommender-system\data\data_with_genre.csv')

In [7]:
def download_image(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            return img
        else:
            return None
    except Exception as e:
        print(f"Error downloading image from {url}: {e}")
        return None

In [8]:
df['thumbnail_image'] = df['thumbnail_link'].apply(download_image)

In [9]:
df.to_pickle(r'C:\Users\tarun\Desktop\song-recommender-system\data\final_dataset.pkl')